In [ ]:
!pip install datasets
!pip install transformers
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00


In [ ]:
import string
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import jamspell
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import torch
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, set_seed

from sklearn.metrics import classification_report, f1_score
from scipy.special import softmax
from tqdm import tqdm

import datasets
from datasets import load_dataset

In [ ]:
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

In [ ]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.']])

def light_clean(x):
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Signos de puntuacion menos. 
  x = x.translate(str.maketrans('', '', punct))
  x = x.replace('#', '')
  # Spell check
  x = corrector.FixFragment(x)
  return x

In [ ]:
# vectorizamos la funcion utilizada para limpiar el texto.
clean1_vect = np.vectorize(light_clean)

In [ ]:
# Aplicamos la funcion en el training, validation y test set.
data = dataset.map(lambda e: {"text_clean": clean1_vect(e['text'])}, batched=True)

In [ ]:
# Modelo a utilizar
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
# Iniciamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True, max_length=512)
#config = AutoConfig.from_pretrained(MODEL)

In [ ]:
# Inicializamos nuestro modelo.
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
#tokenizamos 
data = data.map(lambda e: tokenizer(e['text'], truncation=True, max_length=512), batched=True)

In [ ]:
# Sets en Formato Dict, datos listos para darselos al modelo.
train_dataset = data['train']
val_dataset = data['validation']
test_dataset = data['test']

In [ ]:
# Class distribution -> Unbalanced data.
train_df = pd.DataFrame(dataset['train'])
train_df['label'].value_counts(normalize=True).sort_index()

In [ ]:
class_weights = (1 - (train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")

In [ ]:
train_dataset = train_dataset.rename_column('label', 'labels')

In [ ]:
class WeightLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    # feed inputs to the model and extract logits
    outputs = model(**inputs)
    logits = outputs.get('logits')
    # extract label
    labels = inputs.get('labels')
    # define loss function with lass weights
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    # compute loss
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [ ]:
modelFineTuned = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
batch_size = 64
logging_steps = len(data['train']) // batch_size
training_args = TrainingArguments(
    output_dir='roberta-sentiment-analysis-finetune',
    num_train_epochs=10, 
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    warmup_steps = 50,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=logging_steps,
    load_best_model_at_end = True,
    fp16= True,
    push_to_hub=True
)

In [ ]:
trainer = Trainer(model=modelFineTuned,
                            args=training_args,
                            train_dataset = train_dataset,
                            eval_dataset=val_dataset,
                            tokenizer=tokenizer)

In [ ]:
trainer.train()

In [ ]:
test_preds_raw, test_labels , _ = trainer.predict(test_dataset)
test_preds = np.argmax(test_preds_raw, axis=-1)
print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
trainer.push_to_hub("roberta-sentiment-analysis-finetune")